ADT Pattern in Python
- https://github.com/sabrinahu5/program-synthesis/blob/main/interpreter/flashfill-interpreter/ff-interpreter.py

In [1]:
from pprint import pprint
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict

In [2]:
class ImmutableAssignDict(dict):
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(f"Cannot overwrite existing key: {key}")
        super().__setitem__(key, value)

@dataclass(frozen=True)
class LType:
    pass

@dataclass(frozen=True)
class LBool(LType):
    pass

@dataclass(frozen=True)
class LInt(LType):
    pass

@dataclass(frozen=True)
class LNilType(LType):
    pass

@dataclass(frozen=True)
class LArrow(LType):
    param_type: LType
    return_type: LType

@dataclass(frozen=True)
class LProduct(LType):
    left: LType
    right: LType

@dataclass(frozen=True)
class LSum(LType):
    left: LType
    right: LType

@dataclass(frozen=True)
class LTerm:
    pass

@dataclass(frozen=True)
class LBegin(LTerm):
    terms: list[LTerm]

@dataclass(frozen=True)
class LVar(LTerm):
    name: str

@dataclass(frozen=True)
class LAbs(LTerm):
    param: LVar
    param_type: LType
    body: LTerm

@dataclass(frozen=True)
class LApp(LTerm):
    func: LTerm
    arg: LTerm

@dataclass(frozen=True)
class LClosure(LTerm):
    param: LVar
    param_type: LType
    body: LTerm
    env: ImmutableAssignDict[LVar, LTerm]

@dataclass(frozen=True)
class LTrue(LTerm):
    pass

@dataclass(frozen=True)
class LFalse(LTerm):
    pass

@dataclass(frozen=True)
class LIf(LTerm):
    condition: LTerm
    then_branch: LTerm
    else_branch: LTerm

@dataclass(frozen=True)
class LDefine(LTerm):
    name: LVar
    body: LTerm

@dataclass(frozen=True)
class LList(LType):
    element_type: LType

@dataclass(frozen=True)
class LNil(LTerm):
    pass

@dataclass(frozen=True)
class LCons(LTerm):
    car: LTerm
    cdr: LTerm

@dataclass(frozen=True)
class LNum(LTerm):
    value: int

@dataclass(frozen=True)
class LAdd(LTerm):
    left: LTerm
    right: LTerm

@dataclass(frozen=True)
class LCar(LTerm):
    lst: LTerm

@dataclass(frozen=True)
class LCdr(LTerm):
    lst: LTerm

TypeEnv = ImmutableAssignDict[LVar, LType]
EvalEnv = ImmutableAssignDict[LVar, LTerm]

Early solution for dealing with variable capture by traversing the AST in a preprocessing step and making sure variable capture is impossible. I later realized this was unnecessary because I'm evaluating all variables to values before I run the program.

In [ ]:
import itertools

_counter = itertools.count()

def fresh_var(prefix="v"):
    """Generate a fresh variable name."""
    return f"_{prefix}{next(_counter)}"

def alpha_rename(term, env=None, mapping=None):
    """Recursively rename all variables in the term to ensure uniqueness."""
    if env is None:
        env = ImmutableAssignDict()
    if mapping is None:
        mapping = ImmutableAssignDict()

    if isinstance(term, LVar):
        # If the variable is in the renaming environment, rename it
        if term.name in env:
            return LVar(env[term.name]), mapping
        return term, mapping

    elif isinstance(term, LAbs):
        # Generate a fresh variable name for the parameter
        new_param = fresh_var(term.param.name)
        new_env = env.copy()
        new_env[term.param.name] = new_param  # Add the old name -> new name to the environment

        # Add the mapping from new variable back to the original for potential reversal
        mapping[new_param] = term.param.name

        # Rename the body using the updated environment
        renamed_body, updated_mapping = alpha_rename(term.body, new_env, mapping)

        return LAbs(param=LVar(new_param), param_type=term.param_type, body=renamed_body), updated_mapping

    elif isinstance(term, LApp):
        # Recursively rename both the function and the argument
        renamed_func, mapping = alpha_rename(term.func, env, mapping)
        renamed_arg, mapping = alpha_rename(term.arg, env, mapping)
        return LApp(renamed_func, renamed_arg), mapping

    elif isinstance(term, LIf):
        # Recursively rename condition, then-branch, and else-branch
        renamed_condition, mapping = alpha_rename(term.condition, env, mapping)
        renamed_then, mapping = alpha_rename(term.then_branch, env, mapping)
        renamed_else, mapping = alpha_rename(term.else_branch, env, mapping)
        return LIf(renamed_condition, renamed_then, renamed_else), mapping

    elif isinstance(term, LCons):
        # Recursively rename car and cdr
        renamed_car, mapping = alpha_rename(term.car, env, mapping)
        renamed_cdr, mapping = alpha_rename(term.cdr, env, mapping)
        return LCons(renamed_car, renamed_cdr), mapping

    elif isinstance(term, LDefine):
        # Generate a fresh name for the variable being defined
        new_name = fresh_var(term.name.name)
        new_env = env.copy()
        new_env[term.name.name] = new_name
        mapping[new_name] = term.name.name

        # Rename the body of the definition
        renamed_body, updated_mapping = alpha_rename(term.body, new_env, mapping)
        return LDefine(LVar(new_name), renamed_body), updated_mapping

    elif isinstance(term, LAdd):
        # Recursively rename both sides of the addition
        renamed_left, mapping = alpha_rename(term.left, env, mapping)
        renamed_right, mapping = alpha_rename(term.right, env, mapping)
        return LAdd(renamed_left, renamed_right), mapping

    elif isinstance(term, LCar):
        renamed_lst, mapping = alpha_rename(term.lst, env, mapping)
        return LCar(renamed_lst), mapping

    elif isinstance(term, LCdr):
        renamed_lst, mapping = alpha_rename(term.lst, env, mapping)
        return LCdr(renamed_lst), mapping

    elif isinstance(term, LBegin):
        # Recursively rename all terms in the begin sequence
        renamed_terms = []
        for t in term.terms:
            renamed_t, mapping = alpha_rename(t, env, mapping)
            renamed_terms.append(renamed_t)
        return LBegin(renamed_terms), mapping

    else:
        # No renaming needed for literals (LNum, LTrue, LFalse, etc.)
        return term, mapping

def revert_renaming(term, mapping):
    """Recursively revert renamed variables to their original names using the mapping."""
    if isinstance(term, LVar):
        # If the variable was renamed, revert it back to the original
        if term.name in mapping:
            return LVar(mapping[term.name])
        return term
    elif isinstance(term, LAbs):
        # Revert the parameter name and recurse into the body
        original_param = mapping.get(term.param.name, term.param.name)
        reverted_body = revert_renaming(term.body, mapping)
        return LAbs(LVar(original_param), term.param_type, reverted_body)
    elif isinstance(term, LApp):
        reverted_func = revert_renaming(term.func, mapping)
        reverted_arg = revert_renaming(term.arg, mapping)
        return LApp(reverted_func, reverted_arg)
    elif isinstance(term, LClosure):
        # Revert the closure environment as well as the body and param
        original_param = mapping.get(term.param.name, term.param.name)
        reverted_body = revert_renaming(term.body, mapping)
        reverted_env = {revert_renaming(k, mapping): revert_renaming(v, mapping) for k, v in term.env.items()}
        return LClosure(LVar(original_param), term.param_type, reverted_body, reverted_env)
    elif isinstance(term, LIf):
        reverted_condition = revert_renaming(term.condition, mapping)
        reverted_then = revert_renaming(term.then_branch, mapping)
        reverted_else = revert_renaming(term.else_branch, mapping)
        return LIf(reverted_condition, reverted_then, reverted_else)
    elif isinstance(term, LCons):
        reverted_car = revert_renaming(term.car, mapping)
        reverted_cdr = revert_renaming(term.cdr, mapping)
        return LCons(reverted_car, reverted_cdr)
    elif isinstance(term, LAdd):
        reverted_left = revert_renaming(term.left, mapping)
        reverted_right = revert_renaming(term.right, mapping)
        return LAdd(reverted_left, reverted_right)
    elif isinstance(term, LCar):
        reverted_lst = revert_renaming(term.lst, mapping)
        return LCar(reverted_lst)
    elif isinstance(term, LCdr):
        reverted_lst = revert_renaming(term.lst, mapping)
        return LCdr(reverted_lst)
    elif isinstance(term, LBegin):
        reverted_terms = [revert_renaming(t, mapping) for t in term.terms]
        return LBegin(reverted_terms)
    else:
        return term  # For literals like LNum, LTrue, LFalse, etc.

Typechecking and evaluation

In [43]:
def typecheck(gamma_in: TypeEnv, term: LTerm) -> LType:
    gamma = gamma_in.copy()
    if isinstance(term, LVar):
        if term in gamma:
            return gamma[term]
        else:
            raise RuntimeError(f"Unbound variable error: {term.name}")
    elif isinstance(term, LNil):
        return LNilType()
    elif isinstance(term, LNum):
        return LInt()
    elif isinstance(term, LTrue) or isinstance(term, LFalse):
        return LBool()
    elif isinstance(term, LAbs):
        param_type = term.param_type
        gamma[term.param] = param_type
        return LArrow(param_type, typecheck(gamma, term.body))
    elif isinstance(term, LApp):
        func_type = typecheck(gamma, term.func)
        arg_type = typecheck(gamma, term.arg)
        if isinstance(func_type, LArrow) and func_type.param_type == arg_type:
            return func_type.return_type
        else:
            raise RuntimeError(f"Application type error: Expected {func_type.param_type}, but got {arg_type}")
    elif isinstance(term, LIf):
        condition_type = typecheck(gamma, term.condition)
        then_type = typecheck(gamma, term.then_branch)
        else_type = typecheck(gamma, term.else_branch)
        if isinstance(condition_type, LBool) and then_type == else_type:
            return else_type
        else:
            raise RuntimeError(f"Conditional type error: {term}")
    elif isinstance(term, LCons):
        car_type = typecheck(gamma, term.car)
        cdr_type = typecheck(gamma, term.cdr)
        if isinstance(cdr_type, LNilType):
            return LList(car_type)  # Infer the type of LNil from the car

        if isinstance(cdr_type, LList) and cdr_type.element_type == car_type:
            return LList(car_type)
        else:
            raise RuntimeError("Type mismatch in cons cell")
    elif isinstance(term, LDefine):
        body_type = typecheck(gamma, term.body)
        gamma[term.name] = body_type
        return body_type
    elif isinstance(term, LAdd):
        left_type = typecheck(gamma, term.left)
        right_type = typecheck(gamma, term.right)
        if isinstance(left_type, LInt) and isinstance(right_type, LInt):
            return LInt()
        else:
            raise RuntimeError(f"Addition type error: {term}")
    elif isinstance(term, LCar):
        lst_type = typecheck(gamma, term.lst)
        if isinstance(lst_type, LList):
            return lst_type.element_type
        else:
            raise RuntimeError(f"Car applied to a non-list: {term}")
    elif isinstance(term, LCdr):
        lst_type = typecheck(gamma, term.lst)
        if isinstance(lst_type, LList):
            return LList(lst_type.element_type)
        else:
            raise RuntimeError(f"Cdr applied to a non-list: {term}")
    elif isinstance(term, LBegin):
        for t in term.terms[:-1]:
            typecheck(gamma, t)
        return typecheck(gamma, term.terms[-1])
    else:
        raise RuntimeError(f"Unknown type case: {term}")

def closure_to_lambda(term):
    """Recursively convert closures back to their lambda expression form."""
    if isinstance(term, LClosure):
        # Convert the closure back to LAbs, ignoring the environment
        return LAbs(param=term.param, param_type=term.param_type, body=closure_to_lambda(term.body))
    elif isinstance(term, LApp):
        # Convert function and argument recursively
        return LApp(closure_to_lambda(term.func), closure_to_lambda(term.arg))
    elif isinstance(term, LIf):
        return LIf(closure_to_lambda(term.condition),
                   closure_to_lambda(term.then_branch),
                   closure_to_lambda(term.else_branch))
    elif isinstance(term, LCons):
        return LCons(closure_to_lambda(term.car), closure_to_lambda(term.cdr))
    elif isinstance(term, LAdd):
        return LAdd(closure_to_lambda(term.left), closure_to_lambda(term.right))
    elif isinstance(term, LCar):
        return LCar(closure_to_lambda(term.lst))
    elif isinstance(term, LCdr):
        return LCdr(closure_to_lambda(term.lst))
    elif isinstance(term, LBegin):
        return LBegin([closure_to_lambda(t) for t in term.terms])
    else:
        return term  # Return simple values like LVar, LNum, LTrue, LFalse, etc.

def run_program(initial_env, ast, alpha_conversion=False):
    if alpha_conversion:
        renamed_ast, rename_mapping = alpha_rename(ast)
        env, result = evaluate(initial_env, renamed_ast)
        result = revert_renaming(result, rename_mapping)
        result = closure_to_lambda(result)
    else:
        env, result = evaluate(initial_env, ast)
        result = closure_to_lambda(result)
    return env, result

def substitute(body, param, arg):
    """
    Substitute the parameter `param` with the argument `arg` in the `body`.
    This recursively replaces every occurrence of `param` with `arg`.
    
    Parameters:
    - body: The term in which to perform the substitution.
    - param: The parameter (variable) to replace.
    - arg: The argument to substitute in place of the parameter.
    """
    if isinstance(body, LVar):
        # If the variable matches the parameter, replace it with the argument
        if body.name == param.name:
            return arg
        else:
            return body
    
    elif isinstance(body, LAbs):
        # If the lambda's bound variable matches the parameter, don't substitute within its scope
        if body.param.name == param.name:
            return body
        else:
            # Substitute recursively in the body of the lambda
            return LAbs(body.param, body.param_type, substitute(body.body, param, arg))
    
    elif isinstance(body, LApp):
        # Substitute in both the function and the argument of the application
        new_func = substitute(body.func, param, arg)
        new_arg = substitute(body.arg, param, arg)
        return LApp(new_func, new_arg)
    
    elif isinstance(body, LIf):
        # Substitute in the condition, then-branch, and else-branch
        new_condition = substitute(body.condition, param, arg)
        new_then = substitute(body.then_branch, param, arg)
        new_else = substitute(body.else_branch, param, arg)
        return LIf(new_condition, new_then, new_else)
    
    elif isinstance(body, LCons):
        # Substitute in both car and cdr of the list
        new_car = substitute(body.car, param, arg)
        new_cdr = substitute(body.cdr, param, arg)
        return LCons(new_car, new_cdr)
    
    elif isinstance(body, LAdd):
        # Substitute in both sides of an addition expression
        new_left = substitute(body.left, param, arg)
        new_right = substitute(body.right, param, arg)
        return LAdd(new_left, new_right)

    elif isinstance(body, LBegin):
        # Substitute in all the terms in a sequence (begin)
        new_terms = [substitute(term, param, arg) for term in body.terms]
        return LBegin(new_terms)

    # For literals like LNum, LTrue, LFalse, return them as is
    return body

def evaluate(env_in: EvalEnv, term: LTerm) -> tuple[EvalEnv, LTerm]:
    env = env_in.copy()  # Work on a copy of the environment
    if isinstance(term, LAbs):
        return env, LClosure(term.param, term.param_type, term.body, env)
    elif isinstance(term, LClosure):
        return env, term
    elif isinstance(term, LVar):
        if term in env:
            return env, env[term]
        else:
            raise RuntimeError(f"Unbound variable error: {term.name}")
    elif isinstance(term, LIf):
        updated_env, condition = evaluate(env, term.condition)
        if isinstance(condition, LTrue):
            return evaluate(updated_env, term.then_branch)
        elif isinstance(condition, LFalse):
            return evaluate(updated_env, term.else_branch)
        else:
            raise RuntimeError("Condition is supposed to only be true or false")
    elif isinstance(term, LApp):
        # Step 1: Evaluate the function (should result in a closure)
        _, reduced_func = evaluate(env, term.func)
        
        # Step 2: Evaluate the argument
        _, reduced_arg = evaluate(env, term.arg)

        # Step 3: Ensure the function is a closure
        if not isinstance(reduced_func, LClosure):
            raise RuntimeError("Attempting to apply a non-closure")

        # Step 4: Perform substitution of the argument for the parameter
        substituted_body = substitute(reduced_func.body, reduced_func.param, reduced_arg)

        # Step 5: Evaluate the substituted body in the closure's captured environment
        return evaluate(reduced_func.env, substituted_body)
    elif isinstance(term, LDefine):
        updated_env, evaluated_body = evaluate(env, term.body)
        updated_env[term.name] = evaluated_body
        return updated_env, evaluated_body
    elif isinstance(term, LBegin):
        for subterm in term.terms[:-1]:
            updated_env, _ = evaluate(env, subterm)
            env = updated_env
        return evaluate(env, term.terms[-1])
    elif isinstance(term, LAdd):
        _, left_val = evaluate(env, term.left)
        _, right_val = evaluate(env, term.right)
        if isinstance(left_val, LNum) and isinstance(right_val, LNum):
            return env, LNum(left_val.value + right_val.value)
        else:
            raise RuntimeError("Runtime error in addition: non-numeric values")
    elif isinstance(term, LCons):
        _, car_val = evaluate(env, term.car)
        _, cdr_val = evaluate(env, term.cdr)
        return env, LCons(car_val, cdr_val)
    elif isinstance(term, LCar):
        _, lst_val = evaluate(env, term.lst)
        if isinstance(lst_val, LCons):
            return env, lst_val.car
        else:
            raise RuntimeError("Car operation on non-cons cell")
    elif isinstance(term, LCdr):
        _, lst_val = evaluate(env, term.lst)
        if isinstance(lst_val, LCons):
            return env, lst_val.cdr
        else:
            raise RuntimeError("Cdr operation on non-cons cell")
    elif isinstance(term, LNil) or isinstance(term, LNum) or isinstance(term, LTrue) or isinstance(term, LFalse):
        return env, term
    else:
        raise RuntimeError(f"Evaluation error: {term}")

# TEST CASES

In [44]:
x_var = LVar("x")
y_var = LVar("y")

# First lambda function: λx. λy. x (ignores y and returns x)
first_lambda = LAbs(
    param=x_var, 
    param_type=LInt(),  # For example, assume x is an integer
    body=LAbs(
        param=y_var, 
        param_type=LInt(),  # Assume y is an integer too
        body=LVar("x")  # Returns x
    )
)

# Second lambda function: λy. y (identity function for y)
second_lambda = LAbs(
    param=y_var,
    param_type=LInt(),  # Assume y is an integer
    body=LVar("y")  # Returns y
)

# Application: (λx. λy. x) (λy. y) = (λy. λy. y)
app = LApp(func=first_lambda, arg=second_lambda)

env = ImmutableAssignDict()
_, result = run_program(env, app)
pprint(result)

LAbs(param=LVar(name='y'),
     param_type=LInt(),
     body=LAbs(param=LVar(name='y'), param_type=LInt(), body=LVar(name='y')))


In [45]:
# Corrected outer_lambda that applies both x and y
outer_lambda = LAbs(
    param=x_var,
    param_type=LInt(),  # Assume x is an integer
    body=LApp(
        func=LAbs(
            param=y_var,
            param_type=LInt(),  # Assume y is an integer
            body=LAdd(LVar("x"), LVar("y"))  # x + y
        ),
        arg=LNum(5)  # Apply a concrete value to y, e.g., 5
    )
)

# Apply outer_lambda to the argument, e.g., x = 3
application = LApp(
    func=outer_lambda,  # Apply the outer lambda
    arg=LNum(3)  # Pass x = 3 as an argument
)

# Run the program with an empty environment
env = ImmutableAssignDict()
_, result = run_program(env, application)

# Print the result of evaluating the lambda application
pprint(result)

LNum(value=8)


In [46]:
# x and y variable references
x_var = LVar("x")
y_var = LVar("y")

# Inner lambda: λy. x (should refer to the outer x)
inner_lambda = LAbs(
    param=y_var,
    param_type=LInt(),  # Assume y is an integer
    body=LVar("x")  # Refers to x from the outer scope
)

# Outer lambda: λx. (λy. x) applied to some argument
outer_lambda = LAbs(
    param=x_var,  # This binds the outer x
    param_type=LInt(),
    body=LApp(
        func=inner_lambda,  # Apply the inner lambda λy. x
        arg=LNum(42)  # Pass some argument to y (irrelevant in this case)
    )
)

# Apply the outer lambda to another function (which could cause variable capture)
application = LApp(
    func=outer_lambda,  # Apply outer_lambda to an argument
    arg=LNum(3)  # Pass the argument x = 3 to the outer lambda
)

# Run the program with an empty environment
env = ImmutableAssignDict()
_, result = run_program(env, application)

# Print the result of evaluating the lambda application
pprint(result)

LNum(value=3)


In [5]:
# Constants
TRUE = LTrue()
FALSE = LFalse()

# Example terms
identity_bool = LAbs(LVar("x"), LBool(), LVar("x"))
applied_identity = LApp(identity_bool, TRUE)

complex_term_no_type_check = LIf(
    LApp(LAbs(LVar("x"), LBool(), LVar("x")), TRUE),
    LAbs(LVar("y"), LBool(), LVar("y")),
    FALSE
)

complex_term = LIf(
    LApp(LAbs(LVar("x"), LBool(), LVar("x")), TRUE),
    LAbs(LVar("x"), LBool(), LVar("x")),
    LAbs(LVar("y"), LBool(), LVar("y"))
)

In [6]:
pprint(complex_term)

LIf(condition=LApp(func=LAbs(param=LVar(name='x'),
                             param_type=LBool(),
                             body=LVar(name='x')),
                   arg=LTrue()),
    then_branch=LAbs(param=LVar(name='x'),
                     param_type=LBool(),
                     body=LVar(name='x')),
    else_branch=LAbs(param=LVar(name='y'),
                     param_type=LBool(),
                     body=LVar(name='y')))


In [7]:
gamma: TypeEnv = TypeEnv()
gamma[LVar("x")] = LBool()
basic_term = LVar("x")
a = LAbs(LVar("y"), LBool(), LVar("x"))
b = LApp(a, TRUE)
print(typecheck(gamma, complex_term))
print(typecheck(gamma, basic_term))
pprint(typecheck(gamma, a))
pprint(typecheck(gamma, b))

LArrow(param_type=LBool(), return_type=LBool())
LBool()
LArrow(param_type=LBool(), return_type=LBool())
LBool()


In [8]:
env: EvalEnv = EvalEnv()
x = LVar("x")
env[x] = TRUE
complexer_term = LApp(LIf(
    LApp(LAbs(LVar("x"), LBool(), LVar("x")), TRUE),
    LAbs(LVar("x"), LBool(), LVar("x")),
    LAbs(LVar("y"), LBool(), LVar("y"))
), LVar("x"))
print(evaluate(env, complexer_term))

({LVar(name='x'): LTrue()}, LTrue())


In [9]:
list_example = LCons(
    car=LNum(1),
    cdr=LCons(
        car=LNum(2),
        cdr=LCons(
            car=LNum(3),
            cdr=LNil(),
        ),
    ),
)
list_example

LCons(car=LNum(value=1), cdr=LCons(car=LNum(value=2), cdr=LCons(car=LNum(value=3), cdr=LNil())))

In [10]:
typecheck(gamma, list_example), evaluate(ImmutableAssignDict(), list_example)

(LList(element_type=LInt()),
 ({},
  LCons(car=LNum(value=1), cdr=LCons(car=LNum(value=2), cdr=LCons(car=LNum(value=3), cdr=LNil())))))

In [11]:
func_define = LDefine(
    name=LVar("increment"),
    body=LAbs(
        param=LVar("x"),
        param_type=LInt(),
        body=LAdd(LVar("x"), LNum(1))
    )
)

func_application = LApp(
    func=LVar("increment"),
    arg=LNum(5)
)

begin_expr = LBegin([
    func_define,
    func_application
])

gamma = ImmutableAssignDict()
env = ImmutableAssignDict()

print(typecheck(gamma, func_define))  # Should return LArrow(LInt(), LInt())

res_env, res_val = evaluate(env, begin_expr)  # Apply `increment` to 5

print(res_val)  # Should output LNum(6)

LArrow(param_type=LInt(), return_type=LInt())
LNum(value=6)


In [12]:
# Lambda that captures the value of x in its environment
x = LVar("x")
# ((λx. (λy. x + y))(10))(5) => 10 + 5 = 15
# Expected result: the closure captures x = 10, and adds 5 to it
closure_example = LApp(
    LAbs(x, LInt(), LApp(LAbs(LVar("y"), LInt(), LAdd(x, LVar("y"))), LNum(5))),
    LNum(10)
)

env: EvalEnv = EvalEnv()
x = LVar("x")
env[x] = TRUE
res_env, res_val = evaluate(env, closure_example)
print(res_val)  # Output should be LNum(15)

LNum(value=15)


In [13]:
# Define a list [1, 2, 3] using LCons and LNil
list_var = LVar("myList")
define_list = LDefine(
    list_var,
    LCons(LNum(1), LCons(LNum(2), LCons(LNum(3), LNil())))
)

# Define a function that returns a fixed number, ignoring the list
# Function: λx. 42
simple_fn_var = LVar("simple_fn")
simple_fn = LDefine(
    simple_fn_var,
    LAbs(
        LVar("lst"),
        LList(LInt()),  # The input type is a list of integers
        LNum(42)  # The function always returns 42, ignoring its input
    )
)

# Begin expression to define the list and then apply the function to it
begin_expr = LBegin(
    [
        define_list,  # Define myList = [1, 2, 3]
        simple_fn,  # Define the simple function
        LApp(simple_fn_var, list_var)  # Apply simple_fn to myList
    ]
)

# Evaluation and expected result: 42
_, result = evaluate(ImmutableAssignDict(), begin_expr)
print(result)  # Output should be LNum(42)

LNum(value=42)


In [19]:
# Define a list [1, 2, 3] using LCons and LNil
list_var = LVar("myList")
define_list = LDefine(
    list_var,
    LCons(LNum(1), LCons(LNum(2), LCons(LNum(3), LNil())))
)

# Define the LCar operation to get the first element of the list
car_expr = LCar(list_var)

# Define the LCdr operation to get the tail of the list
cdr_expr = LCdr(list_var)

# Begin expression to define the list and retrieve the head and tail
begin_car_expr = LBegin(
    [
        define_list,   # Define myList = [1, 2, 3]
        car_expr      # Get the head of the list (should be 1)
    ]
)

begin_cdr_expr = LBegin(
    [
        define_list,   # Define myList = [1, 2, 3]
        cdr_expr       # Get the tail of the list (should be [2, 3])
    ]
)

# Evaluate and check the result
_, car_result = evaluate(ImmutableAssignDict(), begin_car_expr)
print("Head of the list:", car_result)  # Output should be LNum(1), the head of the list

_, cdr_result = evaluate(ImmutableAssignDict(), begin_cdr_expr)
print("Tail of the list:", cdr_result)  # Output should be LCons(LNum(2), LCons(LNum(3), LNil())), the tail of the list

Head of the list: LNum(value=1)
Tail of the list: LCons(car=LNum(value=2), cdr=LCons(car=LNum(value=3), cdr=LNil()))


In [15]:
try:
    gamma[list_var] = LList(LInt())
except:
    pass

gamma

{LVar(name='myList'): LList(element_type=LInt())}

In [16]:
typecheck(gamma, begin_cdr_expr)

LList(element_type=LInt())